# Optiver Trading At The Close

[Competition](https://www.kaggle.com/competitions/optiver-trading-at-the-close)

[GitHub Repository](https://github.com/cvaisnor/DNN-Final-Project)

[Google Slides Presentation](https://docs.google.com/presentation/d/1Xc5F1_NveFi1il3GqHej2aqVmklR_jZU4kx6ZJhiDEM/edit?usp=sharing)

# General Imports

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

kaggle_environment = False # True if running on Kaggle, don't forget to add the dataset!

if kaggle_environment:
    data_path = '/kaggle/input/'
else:
    data_path = 'kaggle/input/'

# Setting File Structure

In [2]:
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input/optiver-trading-at-the-close/.gitkeep
kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
kaggle/input/optiver-trading-at-the-close/train.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/.ipynb_checkpoints/test-checkpoint.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/.ipynb_checkpoints/sample_submission-checkpoint.csv
kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so


# Loading Data

In [36]:
original_train = pd.read_csv(data_path + 'optiver-trading-at-the-close/train.csv')
revealed_targets = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/sample_submission.csv')

# Feature Engineering Section

In [37]:
original_train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 16 c

In [38]:
split_ratio = 0.8  # 80% for training, 20% for testing
split_idx = int(len(original_train) * split_ratio)

train = original_train.iloc[:split_idx]
test = original_train.iloc[split_idx:]

y = train['target'].values
y_test = test['target'].values

In [124]:
%%time
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler

class LogFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for column in self.columns:
            X[f'{column}_log'] = np.log1p(X[column].clip(lower=0.00001))
        return X

class WapLagFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, shift_sizes):
        self.shift_sizes = shift_sizes
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for shift_size in self.shift_sizes:
            X[f'wap_lag{shift_size}'] = X.groupby('stock_id')['wap'].shift(shift_size)
        return X

class LagFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, features, shift_sizes):
        self.features = features
        self.shift_sizes = shift_sizes
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for i in range(len(self.features)):
            feature = self.features[i]
            shift_sizes = self.shift_sizes[i]
            for shift_size in self.shift_sizes:
                X[f'{feature}_lag_{shift_size}'] = X.groupby('stock_id')[feature].shift(shift_size)
            return X

class WapRollingMeanFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, window_sizes):
        self.window_sizes = window_sizes
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for window_size in self.window_sizes:
            X[f'wap_rolling_mean{window_size}'] = X.groupby('stock_id')['wap'].rolling(window=10).mean().reset_index(level=0, drop=True)
        return X

class WapDiffFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X['wap_diff'] = X.groupby('stock_id')['wap'].diff()
        return X

class WapExpandingMeanFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X['wap_expanding_mean'] = X.groupby('stock_id')['wap'].expanding().mean().reset_index(level=0, drop=True)
        return X

class ForwardFillValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.fillna(method='ffill', inplace=True)
        return X

class FillZero(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.fillna(0)
        return X

class DataFrameWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, transformer, columns=None):
        self.transformer = transformer
        self.columns = columns
    
    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        return self
    
    def transform(self, X):
        result = self.transformer.transform(X)
        if isinstance(result, pd.DataFrame):
            return result
        if self.columns is None:
            columns = X.columns
        else:
            columns = []
            for column in X.columns:
                if column in self.columns:
                    columns.append(column)
        return pd.DataFrame(result, columns=columns)

class PolynomialFeaturesWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2):
        self.degree = degree
        self.poly = PolynomialFeatures(degree=self.degree, include_bias=False)
        self.new_feature_names = None
    
    def fit(self, X, y=None):
        self.poly.fit(X)
        self.new_feature_names = self.poly.get_feature_names_out(X.columns)
        return self
    
    def transform(self, X):
        transformed_array = self.poly.transform(X)
        return pd.DataFrame(transformed_array, columns=self.new_feature_names)


# Column Preprocessor
columns_to_keep = [
    'imbalance_size_log', 'matched_size_log', 'stock_id', 'imbalance_buy_sell_flag', 'reference_price',
    'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'wap_lag_1',
    'wap_lag_5', # 'wap_lag10', 'wap_lag15', 'wap_lag20',
    'wap_rolling_mean2', 'wap_rolling_mean3',
    'wap_rolling_mean4', 'wap_rolling_mean5', 'wap_rolling_mean6', 'wap_rolling_mean10', 'wap_diff',
    'wap_expanding_mean', 'seconds_in_bucket', 'date_id',
    'matched_size_lag_1', 'matched_size_lag_3', 'matched_size_lag_5'
]
one_hot_cols = []
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), one_hot_cols),
        ('passthrough', 'passthrough', columns_to_keep)
    ],
    remainder='drop'  # Drop other columns
)

# Main Pipeline
pipeline = Pipeline([
    ('logs', LogFeatures(['imbalance_size', 'matched_size'])),
    ('wap_lags', LagFeatures(['wap'], [1, 5])),
    ('matched_size_lags', LagFeatures(['matched_size'], [1, 3, 5])),
    ('wap_rolling_means', WapRollingMeanFeatures([2, 3, 4, 5, 6, 10])),
    ('wap_diff', WapDiffFeature()),
    ('wap_expanding_mean', WapExpandingMeanFeature()),
    ('forward_fill', ForwardFillValues()),
    ('fill_zero', FillZero()),
    ('preprocessor', DataFrameWrapper(preprocessor, columns_to_keep)),
    ('scaler', DataFrameWrapper(StandardScaler())),
    # ('poly', PolynomialFeaturesWrapper(2)),
])

train_transformed = pipeline.fit_transform(train)
test_transformed = pipeline.transform(test)

<timed exec>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

CPU times: user 12.7 s, sys: 2.97 s, total: 15.7 s
Wall time: 16.3 s


In [125]:
X = train_transformed
X_test = test_transformed

In [126]:
# %%time
# train['wap_lag1'] = train.groupby('stock_id')['wap'].shift(1)
# test['wap_lag1'] = test.groupby('stock_id')['wap'].shift(1)

# train['wap_lag5'] = train.groupby('stock_id')['wap'].shift(5)
# test['wap_lag5'] = test.groupby('stock_id')['wap'].shift(5)

# train['wap_rolling_mean10'] = train.groupby('stock_id')['wap'].rolling(window=10).mean().reset_index(level=0, drop=True)
# test['wap_rolling_mean10'] = test.groupby('stock_id')['wap'].rolling(window=10).mean().reset_index(level=0, drop=True)

# train['wap_diff'] = train.groupby('stock_id')['wap'].diff()
# test['wap_diff'] = test.groupby('stock_id')['wap'].diff()

# train['wap_expanding_mean'] = train.groupby('stock_id')['wap'].expanding().mean().reset_index(level=0, drop=True)
# test['wap_expanding_mean'] = test.groupby('stock_id')['wap'].expanding().mean().reset_index(level=0, drop=True)

# train = train.fillna(0)
# test = train.fillna(0)

# train = train.fillna(0)
# test = test.fillna(0)

# train.fillna(method='ffill', inplace=True)
# test.fillna(method='ffill', inplace=True)
# train.fillna(train.mean(), inplace=True)

# drop row_id column
# train = train.drop(columns=['row_id'])
# test = test.drop(columns=['row_id'])



# X = train.drop(columns='target')
# X_test = test.drop(columns='target')

# # drop_columns = ['date_id', 'time_id', 'seconds_in_bucket']
# drop_columns = ['time_id']
# X = X.drop(columns=drop_columns)
# X_test = X_test.drop(columns=drop_columns)

# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# float_columns = [col for col in X.columns if X[col].dtype == 'float64']
# degree2_columns = float_columns + ['stock_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag']
# poly_transformer = ColumnTransformer(
#     transformers=[
#         ('poly2', PolynomialFeatures(degree=(2, 2)), degree2_columns),
#     ],
#     remainder='passthrough'
# )

# pipeline = Pipeline([
#     ('poly_features', poly_transformer)
# ])

# X = pipeline.fit_transform(X)
# X_test = pipeline.transform(X_test)
# scale all X columns that are floats
# float_columns = [col for col in X.columns if X[col].dtype == 'float64']
# scaler = StandardScaler()
# X[float_columns] = scaler.fit_transform(X[float_columns])

# X['wap_lag1'] = X.groupby('stock_id')['wap'].shift(1)
# X_test['wap_lag1'] = X_test.groupby('stock_id')['wap'].shift(1)

# X = X.fillna(0)
# X_test = X_test.fillna(0)


In [128]:
X['wap_lag_1']

0         -1.700446
1         -1.700446
2         -1.700446
3         -1.700446
4         -1.700446
             ...   
4190379    0.882958
4190380    0.945968
4190381    0.945968
4190382    0.945968
4190383    0.945968
Name: wap_lag_1, Length: 4190384, dtype: float64

In [129]:
# drop_columns = ['date_id', 'time_id']
# # drop_columns = []
# X = X.drop(columns=drop_columns)
# X_test = X_test.drop(columns=drop_columns)
# # X = pd.get_dummies(X, columns=['stock_id'], prefix='stock')

All feature transformations should be combined into a generate_feature() function. This can then be applied to the test data will in the for loop for the Optiver API.

In [130]:
def generate_features(cumulative_test_df, current_test):
    cumulative_test_df = pipeline.transform(cumulative_test_df)
    # cumulative_test_df['wap_lag1'] = cumulative_test_df.groupby('stock_id')['wap'].shift(1)
    # cumulative_test_df['wap_lag5'] = cumulative_test_df.groupby('stock_id')['wap'].shift(5)
    # cumulative_test_df['wap_rolling_mean10'] = cumulative_test_df.groupby('stock_id')['wap'].rolling(window=10).mean().reset_index(level=0, drop=True)
    # cumulative_test_df['wap_diff'] = cumulative_test_df.groupby('stock_id')['wap'].diff()
    # cumulative_test_df['wap_expanding_mean'] = cumulative_test_df.groupby('stock_id')['wap'].expanding().mean().reset_index(level=0, drop=True)
    # cumulative_test_df.fillna(method='ffill', inplace=True)
    # cumulative_test_df = cumulative_test_df.drop(columns=['row_id'])
    # cumulative_test_df = cumulative_test_df.fillna(0)
    
    # Only return rows corresponding to the current test dataframe
    return cumulative_test_df, cumulative_test_df.iloc[-len(current_test):]


# Slicing

Generally, don't change the cell below. It is used to slice the data into training and validation sets.

In [131]:
# imports
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X
y_train = y 

In [132]:
X_train.mean()

stock_id                  -1.251119e-15
date_id                    8.110161e-15
seconds_in_bucket         -4.164857e-17
imbalance_buy_sell_flag    7.593802e-17
reference_price            3.211736e-14
far_price                 -1.843614e-15
near_price                 1.726448e-14
bid_price                 -1.103203e-13
bid_size                  -1.552606e-16
ask_price                  1.198471e-13
ask_size                   1.314791e-16
wap                        3.752659e-13
imbalance_size_log         1.078970e-13
matched_size_log          -4.917885e-14
wap_rolling_mean10        -2.936852e-14
wap_diff                  -2.936852e-14
wap_expanding_mean        -2.936852e-14
wap_rolling_mean2         -2.936852e-14
wap_rolling_mean3         -2.936852e-14
wap_rolling_mean4         -2.936852e-14
wap_rolling_mean5         -1.465042e-18
wap_rolling_mean6          8.221094e-13
wap_lag_1                  4.872622e-17
wap_lag_5                  6.333323e-16
matched_size_lag_1         2.056689e-16


In [133]:
# check types of outputs
print('X_train type: ', type(X_train))
print('X_test type: ', type(X_test))
print()
print('y_train type: ', type(y_train))
print('y_test type: ', type(y_test))

X_train type:  <class 'pandas.core.frame.DataFrame'>
X_test type:  <class 'pandas.core.frame.DataFrame'>

y_train type:  <class 'numpy.ndarray'>
y_test type:  <class 'numpy.ndarray'>


In [134]:
# # convert to numpy arrays
# X_train = X_train.values
# X_test = X_test.values

# Model Development

In [135]:
%%time
import lightgbm as lgb
from sklearn.pipeline import Pipeline


# lgbm = lgb.LGBMRegressor(n_jobs=-1, random_state=0, force_col_wise=True,
#                          verbose=-1, boosting_type='gbdt', num_leaves=10,
#                          reg_alpha=0, reg_lambda=0.2, objective='regression_l1')

lgbm = lgb.LGBMRegressor(n_jobs=-1, random_state=0, objective='regression_l1')
lgbm.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6372
[LightGBM] [Info] Number of data points in the train set: 4190384, number of used features: 27
[LightGBM] [Info] Start training from score -0.069737
CPU times: user 1min 38s, sys: 3.43 s, total: 1min 42s
Wall time: 11.3 s


LGBMRegressor(n_jobs=-1, objective='regression_l1', random_state=0)

In [136]:
# lgbm.score(X_test, y_test)

from sklearn.metrics import mean_absolute_error

predictions = lgbm.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error on the test set: {mae:.4f}")
predictions2 = lgbm.predict(X_train)
mae2 = mean_absolute_error(y_train, predictions2)
print(f"Mean Absolute Error on the training set: {mae2:.4f}")


Mean Absolute Error on the test set: 5.9872
Mean Absolute Error on the training set: 6.3652


In [137]:
y_test

array([ 10.470152 , -16.819836 ,  -2.580285 , ...,   1.1694431,
        -1.540184 ,  -6.530285 ])

In [138]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [18]:
print('PyTorch Version: ', torch.__version__)
print('CUDA Available: ', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device Type: ', torch.cuda.get_device_name(0))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

PyTorch Version:  2.1.0
CUDA Available:  False


In [19]:
# convert to numpy arrays
# X_train = X_train.values
# X_test = X_test.values

In [20]:
class PyTorchMLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PyTorchMLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size


        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, hidden_size)  # Output is a single floating point number
        self.fc4 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

    def fit(self, X_train, y_train, X_test, y_test, epochs, batch_size, learning_rate, device, verbose=True):
        self.to(device)
        self.train()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        criterion = nn.L1Loss()
        X_train_t = torch.tensor(X_train, dtype=torch.float32, device=device)
        y_train_t = torch.tensor(y_train, dtype=torch.float32, device=device)
        X_test_t = torch.tensor(X_test, dtype=torch.float32, device=device)
        y_test_t = torch.tensor(y_test, dtype=torch.float32, device=device)
        for epoch in tqdm(range(epochs), desc='Epochs'):
            for i in range(0, len(X_train), batch_size):
                batch_X = X_train_t[i:i+batch_size]
                batch_y = y_train_t[i:i+batch_size]
                optimizer.zero_grad()
                outputs = self(batch_X)
                loss = criterion(outputs, batch_y.view_as(outputs))
                loss.backward()
                optimizer.step()
            self.eval()
            with torch.no_grad():
                y_pred = self(X_test_t)
                test_loss = criterion(y_pred, y_test_t.view_as(y_pred))
                if verbose:
                    print(f'Epoch: {epoch+1}/{epochs}, Loss: {loss.item()}, Test Loss: {test_loss.item()}')
            self.train()

    def predict(self, X_test, device):
        self.to(device)
        self.eval()
        with torch.no_grad():
            X_test_t = torch.tensor(X_test, dtype=torch.float32, device=device)
            y_pred = self(X_test_t)
        return y_pred.cpu().numpy()

In [21]:
# creating the model and fitting it on the data from the original sklearn train test split
# model = PyTorchMLP(input_size=X_train.shape[1], hidden_size=128)
# model.fit(X_train, y_train, X_test, y_test, epochs=2, batch_size=256, learning_rate=0.001, device=device)
model = lgbm

# Generating Predictions

Only for Kaggle:
When uploading notebook, make sure to add the Optiver data on the right side of the screen.

In [22]:
if kaggle_environment:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()

In [23]:
if kaggle_environment:
    # To count how many time the "for loop" runs.
    counter = 0

    # init 3 empty lists
    test_ls, revealed_targets_ls, sample_prediction_ls = [], [], []
    cumulative_test_df = pd.DataFrame()

    for (test_in, revealed_targets, sample_prediction) in iter_test:
        # Append the dataframe that API return into the list.
        test_ls.append(test_in.copy())
        revealed_targets_ls.append(revealed_targets.copy())
        sample_prediction_ls.append(sample_prediction.copy())

        cumulative_test_df = pd.concat([cumulative_test_df, test_in], axis=0, ignore_index=True)

        # Generate features
        (cumulative_test_df, test_in) = generate_features(cumulative_test_df, test_in)

        # Writes our predictions 
        sample_prediction["target"] = model.predict(test_in)
        
        # This line submit our predictions.
        env.predict(sample_prediction)
        counter += 1

    print('\n', '=' * 50, sep="")
    print(f"counter: {counter}")